In [60]:
import nibabel as nib
import numpy as np
import os,sys
import pandas as pd 
import glob
from sklearn.linear_model import LinearRegression
from scipy import stats
#import plotnine as p9
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import statsmodels.api as sm
#from radiomics import featureextractor, getTestCase
#import SimpleITK as sitk
import import_ipynb
sys.path.append('/home/romain/SVN/python/romain')
from Predistim_Library import *
#import radiomics as rm
from scipy import ndimage
from scipy.ndimage import gaussian_filter

In [61]:
CQ_FILE

'/NAS/deathrow/protocoles/predistim/Predistim_MRIdata_v20200406_update20201204.xlsx'

In [59]:
CQData = pd.read_excel(CQ_FILE,index_col=0,engine='openpyxl')
CQData = CQData[CQData.index.notnull()]
CQ_DB=set(CQData.index)

ClinicalData = pd.read_excel(CLI_FILE,index_col=1,engine='openpyxl',converters={'PatientID':str,'PatientNB':str})
ClinicalData = ClinicalData[ClinicalData.index.notnull()]
ClinicalData = ClinicalData[ClinicalData.filter(regex='^(?!Unnamed)').columns]

#ClinicalData = pd.read_excel(CLI_FILE,index_col=1,engine='openpyxl',dtype=str)
#ClinicalData = pd.read_excel(CLI_FILE,index_col=1,engine='openpyxl')

GenomicData = pd.read_excel(SNP_FILE,usecols=('PatientNB','SNP_ACSL4','SNP_GPx4', 'cluster_SNP', 'Selenium', 'Ferritin ', 'GPx ','NFL ', '4HNE ', 'TNFalpha ', 'IL6', 'AlphaSyn ','LEDD_TOT_V0'),index_col=0,engine='openpyxl',converters={'PatientNB':str})
GenomicData = GenomicData[GenomicData.index.notnull()]

Full_DB=set(ClinicalData.set_index(['PatientID']).index)

print(len(CQ_DB | Full_DB))
Full_DB

FileNotFoundError: [Errno 2] No such file or directory: '/NAS/deathrow/protocoles/predistim/Predistim_MRIdata_v20200406_update20201204.xlsx'

In [4]:
CQData.rename(columns={'décision 3DT1\n':'QC3DT1'},inplace=True)
CQData.rename(columns={'décision 3DMULTI':'QCMultiTE'},inplace=True)

In [ ]:
df = pd.DataFrame()

#for ind,SUBJ in enumerate(sorted((CQ_DB | Full_DB) - set(df.index))):
for ind, SUBJ in enumerate(sorted((CQ_DB | Full_DB))):  
    print(SUBJ)
    PARA='3DT1'
    #file = os.path.join(STUDY_PATH, 'HCP', SUBJ, 'T1w', SUBJ, 'mri', 'T1FS_resliced.nii.gz')
    if os.path.isfile(file):
        print(file)                
        #img_data = np.asanyarray(nib.load(file).dataobj)  
        #img_data = np.abs(img_data)
        #img_data = gaussian_filter(img_data, sigma=2.548)
        # VOLBRAIN caudé 3/4, Putamen 5/6, thalamus 7/8, Globus pallidus 9/10, hipocampus 11/12, amigdala 13/14
        struct_file = os.path.join(STUDY_PATH, '3DT1', SUBJ, 'native_lab_' + SUBJ + '_resliced.nii.gz')
        print(struct_file)                        
        if os.path.exists(struct_file):
            list_oar = ["Ventricules", "Caudate", "Putamen", "Thalamus", "Globus_pallidus", "Hippocampus", "Amygde"]                            
            struct = nib.load(struct_file)
            struct_data = np.asanyarray(struct.dataobj, dtype=int)
            voxel_dims = (struct.header["pixdim"])[1] * (struct.header["pixdim"])[2] * (struct.header["pixdim"])[3]
            for cpt1, cpt2, name_struct in zip(np.arange(1, 16, 2), np.arange(2, 17, 2), list_oar):
                df.loc[SUBJ, PARA + "_volume_" + name_struct+"_L"] = np.sum(struct_data == cpt1) * voxel_dims
                df.loc[SUBJ, PARA + "_volume_" + name_struct+"_R"] = np.sum(struct_data == cpt2) * voxel_dims

        else:
            print("Pas de volbrain pour ", SUBJ)
    
        # FREESURFER cingulaireAnterieure 1002&1026/2002&2026 superieurFrontal 1028/2028    
        # struct_file = os.path.join(STUDY_PATH, '3DT1', SUBJ, 'aparc+aseg_resliced.nii.gz')
        struct_file = os.path.join(STUDY_PATH, 'HCP', SUBJ, 'T1w', SUBJ, 'mri', 'aparc+aseg_resliced.nii.gz')
        # print(struct_file)
        if os.path.exists(struct_file): 
            struct = nib.load(struct_file)
            struct_data = np.asanyarray(struct.dataobj)   
            voxel_dims = (struct.header["pixdim"])[1] * (struct.header["pixdim"])[2] * (struct.header["pixdim"])[3]                            
            df.loc[SUBJ, PARA + "_volume_Insula_L"] = np.sum(struct_data == 19) * voxel_dims
            df.loc[SUBJ, PARA + "_volume_Insula_R"] = np.sum(struct_data == 55) * voxel_dims

        else:
            print("Pas de freesurfer pour", SUBJ)
    
        # print("ATLAS_GPe_GPi_STh_STR_RN_SN")
        for cpt2, name_struct in enumerate(['FLASH_RN', 'FLASH_SN', 'QSM_GPe', 'QSM_GPi', 'FLASH_STh', 'MP2RAGE_STR']):
            struct_file_L = os.path.join(STUDY_PATH, '3DT1', SUBJ, name_struct + '_L_' + SUBJ + '_on3DT1.nii.gz')
            struct_file_R = os.path.join(STUDY_PATH, '3DT1', SUBJ, name_struct + '_R_' + SUBJ + '_on3DT1.nii.gz')
            if os.path.exists(struct_file_L) and os.path.exists(struct_file_R): 
                struct_L_ = nib.load(struct_file_L)
                struct_R_ = nib.load(struct_file_R)
                #struct_data = np.asanyarray(struct_R_.dataobj) + np.asanyarray(struct_L_.dataobj)
                voxel_dims = (struct_R_.header["pixdim"])[1] * (struct_R_.header["pixdim"])[2] * (struct_R_.header["pixdim"])[3]
                tmp = (struct_data > 0.5)
                df.loc[SUBJ, PARA + "_volume_" + name_struct+"_L"] = np.sum(struct_L_.get_fdata() > 0.5) * voxel_dims
                df.loc[SUBJ, PARA + "_volume_" + name_struct+"_R"] = np.sum(struct_R_.get_fdata() > 0.5) * voxel_dims
            else:
                print("Pas d'atlas pour", SUBJ, struct_file_L, struct_file_R)

01001SR
/NAS/deathrow/protocoles/predistim/HCP/06020MB/T1w/06020MB/mri/T1FS_resliced.nii.gz
/NAS/deathrow/protocoles/predistim/3DT1/01001SR/native_lab_01001SR_resliced.nii.gz
01002TM
/NAS/deathrow/protocoles/predistim/HCP/06020MB/T1w/06020MB/mri/T1FS_resliced.nii.gz
/NAS/deathrow/protocoles/predistim/3DT1/01002TM/native_lab_01002TM_resliced.nii.gz
01003SJ
/NAS/deathrow/protocoles/predistim/HCP/06020MB/T1w/06020MB/mri/T1FS_resliced.nii.gz
/NAS/deathrow/protocoles/predistim/3DT1/01003SJ/native_lab_01003SJ_resliced.nii.gz
Pas de volbrain pour  01003SJ
Pas de freesurfer pour 01003SJ
Pas d'atlas pour 01003SJ /NAS/deathrow/protocoles/predistim/3DT1/01003SJ/FLASH_RN_L_01003SJ_on3DT1.nii.gz /NAS/deathrow/protocoles/predistim/3DT1/01003SJ/FLASH_RN_R_01003SJ_on3DT1.nii.gz
Pas d'atlas pour 01003SJ /NAS/deathrow/protocoles/predistim/3DT1/01003SJ/FLASH_SN_L_01003SJ_on3DT1.nii.gz /NAS/deathrow/protocoles/predistim/3DT1/01003SJ/FLASH_SN_R_01003SJ_on3DT1.nii.gz
Pas d'atlas pour 01003SJ /NAS/deathrow/p

In [49]:
df.to_csv(os.path.join(STUDY_PATH,'20250303_3DT1_Volumes.csv'))

In [50]:
df.to_excel(os.path.join(STUDY_PATH,'20250303_3DT1_Volumes.xlsx'))

In [51]:
CLI_FILE

'/NAS/deathrow/protocoles/predistim/DataCli_20210104_09122020.xlsx'

In [77]:
# Lire les fichiers Excel
file1 = '/NAS/deathrow/protocoles/predistim/csv_xlsx/Predistim_MRIdata_v20200406_update20201204.xlsx'
file2 = '/NAS/deathrow/protocoles/predistim/20241213-143037_3DT1_rsfMRI_DTI_b0map/Predistim_MRIdata_3DT1_rsfMRI_DTI_b0map_13122024_mod.xlsx'

data1 = pd.read_excel(file1, engine='openpyxl')
data2 = pd.read_excel(file2, engine='openpyxl')

# Extraire les colonnes contenant les codes patients
codes1 = set(data1['Code patient'])
codes2 = set(data2[' Code patient'])

# Comparer les codes patients
common_codes = codes1 & codes2
unique_to_file1 = codes1 - codes2
unique_to_file2 = codes2 - codes1

print(f"Nombre de codes communs: {len(common_codes)}")
print(f"Nombre de codes uniques au fichier 1: {len(unique_to_file1)} {unique_to_file1}")
print(f"Nombre de codes uniques au fichier 2: {len(unique_to_file2)}")

Nombre de codes communs: 777
Nombre de codes uniques au fichier 1: 20 unique_to_file1
Nombre de codes uniques au fichier 2: 14


In [75]:
data2.columns

Index([' Code patient', 'Centre', 'Date d'IRM Dicom', 'commentaire ',
       'constructeur', 'modèle', 'champ', 'logiciel', 'antenne',
       'respect du protocole',
       'QC 3DT1 \n0: inexploitable 1: très  limite   2: passable       3: bien\n4: parfaite',
       'décision 3DT1\n0: rejet\n1: rescan\n3: limite\n4: OK',
       'commentaire 3DT1 qualité (paramètrès)',
       '3DT1 sans correction de distorsion géométrique',
       '3DT1 avec correction de distorsion géométrique'],
      dtype='object')